In [17]:
data_path = './data/VulnExtractData/ffmpeg_test'
train_path = data_path + '/Train'
test_path = data_path + '/Test'
train_path,test_path


('./data/VulnExtractData/ffmpeg_test/Train',
 './data/VulnExtractData/ffmpeg_test/Test')

In [18]:
task_list = ['AF','BF','CL']

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer


def train_model_svm(X_train,Y_train,X_test,Y_test):
    pipeline = make_pipeline(TfidfVectorizer(ngram_range=(1,1)), SVC(probability=True))

    pipeline.fit(X_train,Y_train)

    Y_pred = pipeline.predict(X_test)
    print(pipeline.score(X_test,Y_test))
    #print(classification_report(Y_test, Y_pred, digits=4))

    # Calculate confusion matrix for the binary classification
    cm = confusion_matrix(Y_test, Y_pred)

    # Extracting TP, FP, TN, FN from the confusion matrix
    TP = cm[1, 1]  # True Positives: positive class correctly identified
    FN = cm[1, 0]  # False Negatives: positive class incorrectly marked as negative
    FP = cm[0, 1]  # False Positives: negative class incorrectly marked as positive
    TN = cm[0, 0]  # True Negatives: negative class correctly identified

    # Print TP, FP, TN, FN
    #print(f"True Positives: {TP}")
    #print(f"False Positives: {FP}")
    #print(f"True Negatives: {TN}")
    #print(f"False Negatives: {FN}")

In [ ]:
import os
import pandas as pd

def prepare_data(task_id):    
    folder_train_path = train_path + '/' + task_id

    all_train_files = []
    for root, dirs, files in os.walk(folder_train_path):
        for file in files:
            full_path = os.path.join(root, file)
            all_train_files.append(full_path)

    dfs = [pd.read_csv(file) for file in all_train_files]

    folder_test_path = test_path + '/' + task_id

    all_test_files = []
    for root, dirs, files in os.walk(folder_test_path):
        for file in files:
            full_path = os.path.join(root, file)
            all_test_files.append(full_path)

    dfs_train = [pd.read_csv(file) for file in all_train_files]

    dfs_test = [pd.read_csv(file) for file in all_test_files]

    df_train = pd.concat(dfs_train, ignore_index=True)

    df_test  = pd.concat(dfs_test, ignore_index=True)

    #use only Description for training
    X_train = df_train['Description']
    X_test = df_test['Description']

    Y_train = df_train['label'].apply(lambda x:0 if x==4 else 1).values
    Y_test = df_test['label'].apply(lambda x:0 if x==4 else 1).values

    X_train.shape,X_test.shape,Y_train.shape,Y_test.shape
    return X_train,Y_train,X_test,Y_test

for task_id in task_list:
    X_train,Y_train,X_test,Y_test = prepare_data(task_id)
    #print(X_train.shape,Y_train.shape,X_test.shape,Y_test.shape)
    train_model_svm(X_train,Y_train,X_test,Y_test)





0.8842504743833017
0.9715370018975332
0.9089184060721063


SVM n-gram Model Binary classification

((1897,), (527,), (1897,), (527,))

In [ ]:
model